# Exploratory data analysis : French Second Round election 2017

** Introduction: **

Refer to the readme file on the repo homepage for the background. This notebook explores my proprietary survey on ~1000 youth survey respondents in France to answer the question of where first round votes for neither Macron nor Le Pen were likely to go. 

We find that the data provides support for the hypothesis that a Macron victory is more likely, as young voters who were 'on the fence' in the first round are more likely to switch to Macron than Le Pen. Further, those who think that Le Pen is going to win are more positive on her victory than those who think Macron is likely to win. Perhaps they are more likely to be 'realists'. 

The second notebook in this series looks at some of the textual data generated. 

#### Steps:
1) Pre-upload: download csv file from SurveyMonkey and **anonymize data** by deleting IP and email information (in seperate private notebook). 

2) Simple cleaning the data, renaming axes, dropping unnecessary columns

3) Run descriptive statistics on the data including cross-tabs. 


----------------

## Imports

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', 300)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
from statsmodels.graphics.mosaicplot import mosaic


## Load Data & explore

In [2]:
survey2raw=pd.read_csv("../data/survey2_anonymized.csv", encoding="utf-8", low_memory=False,infer_datetime_format=True,index_col=0)
survey2raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1013
Data columns (total 24 columns):
ID personne interrogée                                                                                                                                                                                                                           1014 non-null int64
ID collecteur                                                                                                                                                                                                                                    1014 non-null int64
Date de début                                                                                                                                                                                                                                    1014 non-null object
Date de fin                                                                                                  

The data has been anonymized prior to uploading by deleting all the values in the columns for IP's and email addresses. 

In [3]:
print("Data shape : %s" %(str(survey2raw.shape)))

Data shape : (1014, 24)


** Change header for clarity**

In [4]:
new_header=['ID', 'ID collecteur', 'Date de début',
       'Date de fin', 'IP', 'Adresse e-mail', 'Prénom', 'Nom',
       'Custom 1', 'Q1', 'Q2','Q3','Q4','Q4-text','Q5', 'Q5-text','Q6','Q6-text',
            'Q7','Q8','Q9','Q10','Q11','Q12']

old_header = survey2raw.columns #store old header just in case
len(new_header) == len(old_header) #check if no column is missing in new header

True

In [5]:
qn_list=pd.DataFrame(old_header,new_header)
qn_list

,0
ID,ID personne interrogée
ID collecteur,ID collecteur
Date de début,Date de début
Date de fin,Date de fin
IP,Adresse IP
Adresse e-mail,Adresse e-mail
Prénom,Prénom
Nom,Nom
Custom 1,Custom 1
Q1,Etes-vous inscrit pour voter?


In [6]:
#substitute old header with new one for clarity
survey2raw.columns=new_header
survey2raw.head(2)

,ID,ID collecteur,Date de début,Date de fin,IP,Adresse e-mail,Prénom,Nom,Custom 1,Q1,Q2,Q3,Q4,Q4-text,Q5,Q5-text,Q6,Q6-text,Q7,Q8,Q9,Q10,Q11,Q12
0,6189761401,156443487,05/09/2017 23:01:30,05/09/2017 23:04:21,NaN,NaN,NaN,NaN,NaN,Non,NaN,Macron,Non,NaN,Cela dépend du candidat,NaN,Emmanuel Macron,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6185182074,156443487,05/06/2017 22:07:12,05/06/2017 22:09:50,NaN,NaN,NaN,NaN,NaN,Oui,NaN,Le Pen,Oui,NaN,Pas sûr,NaN,Marine Le Pen,NaN,Le Pen,On verra,Non,Oui,Oui,Oui


In [7]:
#cast date from string to datetime format
survey2raw['Date de fin'] = pd.to_datetime(survey2raw['Date de fin'])
survey2raw['Date de début'] = pd.to_datetime(survey2raw['Date de début'])

In [8]:
#variable types
summary =  pd.DataFrame()
summary["column"] = list(survey2raw.columns)
summary["type"] = list(survey2raw.dtypes)
summary["nb_missing_values"] = list(survey2raw.isnull().sum())
summary["nb_missing_values%"] = round(summary["nb_missing_values"]/1014*100,2)
summary["unique"]=summary["column"].apply(lambda x : len(set(survey2raw[x])))
summary.set_index("column",inplace=True)
summary


,type,nb_missing_values,nb_missing_values%,unique
column,,,,
ID,int64,0,0.00,1014
ID collecteur,int64,0,0.00,1
Date de début,datetime64[ns],0,0.00,1008
Date de fin,datetime64[ns],0,0.00,1007
IP,float64,1014,100.00,1014
Adresse e-mail,float64,1014,100.00,1014
Prénom,float64,1014,100.00,1014
Nom,float64,1014,100.00,1014
Custom 1,float64,1014,100.00,1014


We note that the following columns are all empty : 
- Adresse e-mail (email adress)
- Prénom (first name)
- Nom (family name)
- Custom 1
- IP (deliberately removed)
- Q2 - emails (deliberately removed)

As you can also see, not every question was answered by 100% of the respondents. I didn't make every single question on the survey compulsory because I did not want people to answer questions they did not care for, and I wanted more responses versus quality of each response.

## Data cleaning

In [9]:
#Delete the four columns that are empty
dff=survey2raw.drop(['Adresse e-mail', 'Prénom', 'Nom',
       'Custom 1','IP','Q2'], axis =1)

In [10]:
dff=dff.fillna("")
#since the input is all object for now I will just use "" as a quick fix 
dff.head(2)

,ID,ID collecteur,Date de début,Date de fin,Q1,Q3,Q4,Q4-text,Q5,Q5-text,Q6,Q6-text,Q7,Q8,Q9,Q10,Q11,Q12
0,6189761401,156443487,2017-05-09 23:01:30,2017-05-09 23:04:21,Non,Macron,Non,,Cela dépend du candidat,,Emmanuel Macron,,,,,,,
1,6185182074,156443487,2017-05-06 22:07:12,2017-05-06 22:09:50,Oui,Le Pen,Oui,,Pas sûr,,Marine Le Pen,,Le Pen,On verra,Non,Oui,Oui,Oui


Save dff to csv for the next notebook that focuses on textual data.

In [11]:
dff.to_csv('../data/survey2_cleaned.csv', sep=',', encoding='utf-8')
#alternatively use % store 

In [12]:
translated_qn={
    'Q1':'Are you registered to vote?',
    'Q3':'Who did you vote for in the first round?',
    'Q4':'Are you planning to vote in the second round??',
    'Q5':'Are you sure of your vote?',
    'Q6':'Macron or Le Pen?',
    'Q7':'Who do you think will win this election?',
    'Q9':'Do your friends think similarly?',
    'Q11':'Do you agree the person you think will win is the right choice for France?',
}

# Exploration

In [13]:
list_qn=["Q1","Q3","Q4","Q5","Q6","Q7","Q9","Q11"]
for variable in list_qn:
    print(variable, '-- ', translated_qn[variable], "-- %s options, normalized %% of total:"\
          %len(set(dff[variable])) )
    print (round(dff[variable].value_counts(normalize=True)*100,2),"\n\n")
   
#Q2 - emails, Q8, Q10, Q12 are long form answers

Q1 --  Are you registered to vote? -- 3 options, normalized % of total:
Oui    77.51
Non    21.79
        0.69
Name: Q1, dtype: float64 


Q3 --  Who did you vote for in the first round? -- 12 options, normalized % of total:
Le Pen           41.62
Melenchon        20.41
Macron           18.84
                  9.57
Hamon             2.96
Fillon            2.17
Poutou            1.68
Dupont-Aignan     1.28
Asselineau        0.49
Lassalle          0.49
Arthaud           0.30
Cheminade         0.20
Name: Q3, dtype: float64 


Q4 --  Are you planning to vote in the second round?? -- 5 options, normalized % of total:
Oui                          72.49
Non                          15.19
Je ne sais pas                7.20
Autre (veuillez préciser)     3.94
                              1.18
Name: Q4, dtype: float64 


Q5 --  Are you sure of your vote? -- 6 options, normalized % of total:
Très sûr                     64.50
Je ne sais pas               11.64
Pas sûr                      10.36
A

# Crosstabs 

In [14]:
other_cand=set(dff['Q3'])
other_cand.remove("Le Pen")
other_cand.remove("Macron")
other_cand.remove("Melenchon")
other_cand.remove("")
print(other_cand)

{'Arthaud', 'Lassalle', 'Hamon', 'Asselineau', 'Poutou', 'Fillon', 'Cheminade', 'Dupont-Aignan'}


In [15]:
mapping = dict()
for key in other_cand:
    mapping[key]="Other"

In [16]:
mapping.update({"Le Pen":"Le Pen","Macron":"Macron", "Melenchon":"Melenchon"})
mapping

{'Arthaud': 'Other',
 'Asselineau': 'Other',
 'Cheminade': 'Other',
 'Dupont-Aignan': 'Other',
 'Fillon': 'Other',
 'Hamon': 'Other',
 'Lassalle': 'Other',
 'Le Pen': 'Le Pen',
 'Macron': 'Macron',
 'Melenchon': 'Melenchon',
 'Poutou': 'Other'}

In [17]:
comparison=["Q3","Q6"]
print(comparison[0],"--",translated_qn[comparison[0]])
print(comparison[1],"--",translated_qn[comparison[1]])

temp = pd.crosstab(index=dff['Q6'],columns=dff['Q3'].replace(mapping), \
                 normalize=True,margins=True).apply(lambda r:r*100, axis=1)\
                 .round(decimals=2)
    
index_all = temp.index.get_loc("All")
array_toReorder = list(temp["All"].argsort())
array_toReorder.reverse()
array_toReorder.remove(index_all)
array_toReorder.append(index_all)
new_index = [temp.index[x] for x in array_toReorder]
temp.reindex(new_index


SyntaxError: unexpected EOF while parsing (<ipython-input-17-14c5c9f4a693>, line 13)

In the first round, we can see that 15.2% reported that they voted for Macron. 15% of respondents say they are not going to vote, which corresponds to question 4. 

If our respondents are representative of all French youth, **the youth do love Marine le Pen! ** Le Pen has twice the support that Macron has among the youth but his 18.8% doesn't look too shabby compared to his national result of 24% in the first round. 

We can also explore the breakdown of the voters' intentions by 1st round vote, by removing Le Pen and Macron: 

In [18]:
comparison=["Q3","Q6"]
print(comparison[0],"--",translated_qn[comparison[0]])
print(comparison[1],"--",translated_qn[comparison[1]])

dff.other=dff[(dff["Q3"]!="Le Pen")&(dff["Q3"]!="Macron")]

temp=pd.crosstab(index=dff.other['Q6'],columns=dff.other['Q3'], \
            normalize=True,margins=True).apply(lambda r:r*100, axis=1)\
            .round(decimals=2)
    
index_all = temp.index.get_loc("All")
array_toReorder = list(temp["All"].argsort())
array_toReorder.reverse()
array_toReorder.remove(index_all)
array_toReorder.append(index_all)
new_index = [temp.index[x] for x in array_toReorder]
temp.reindex(new_index)

Q3 -- Who did you vote for in the first round?
Q6 -- Macron or Le Pen?


Q3,,Arthaud,Asselineau,Cheminade,Dupont-Aignan,Fillon,Hamon,Lassalle,Melenchon,Poutou,All
Q6,,,,,,,,,,,
Emmanuel Macron,6.23,0.25,0.00,0.00,0.25,3.24,4.49,0.00,23.19,1.50,39.15
Blanc,6.23,0.25,1.00,0.25,1.00,0.25,1.50,0.50,16.71,1.00,28.68
Marine Le Pen,7.73,0.25,0.25,0.25,2.00,2.00,1.50,0.75,10.72,1.75,27.18
,3.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,4.99
All,24.19,0.75,1.25,0.50,3.24,5.49,7.48,1.25,51.62,4.24,100.00


We take the preferences of these non-Macron non-Le Pen voters, calculated below, and plug them into my election model, yielding a victory for Macron of 53.2%. 

We should expect an even higher margin of victory, because we may be double-counting abstaining votes from the first round. 

In [19]:
temp=pd.crosstab(index=dff.other['Q6'],columns=dff.other['Q3'], \
            normalize='columns',margins=True).apply(lambda r:r*100)\
            .round(decimals=2)

temp

Q3,,Arthaud,Asselineau,Cheminade,Dupont-Aignan,Fillon,Hamon,Lassalle,Melenchon,Poutou,All
Q6,,,,,,,,,,,
,16.49,0.00,0.0,0.0,0.00,0.00,0.0,0.0,1.93,0.00,4.99
Blanc,25.77,33.33,80.0,50.0,30.77,4.55,20.0,40.0,32.37,23.53,28.68
Emmanuel Macron,25.77,33.33,0.0,0.0,7.69,59.09,60.0,0.0,44.93,35.29,39.15
Marine Le Pen,31.96,33.33,20.0,50.0,61.54,36.36,20.0,60.0,20.77,41.18,27.18


In [20]:
comparison=["Q5","Q6"]
print(comparison[0],"--",translated_qn[comparison[0]])
print(comparison[1],"--",translated_qn[comparison[1]])
temp=pd.crosstab(index=dff['Q6'].replace({"Blanc":""}),columns=dff['Q5'].replace({"":"Autre (veuillez préciser)"}), \
            normalize=True,margins=True).apply(lambda r:r*100)\
            .round(decimals=2)
   
index_all = temp.index.get_loc("All")
array_toReorder = list(temp["All"].argsort())
array_toReorder.reverse()
array_toReorder.remove(index_all)
array_toReorder.append(index_all)
new_index = [temp.index[x] for x in array_toReorder]
temp.reindex(new_index)

Q5 -- Are you sure of your vote?
Q6 -- Macron or Le Pen?


Q5,Autre (veuillez préciser),Cela dépend du candidat,Je ne sais pas,Pas sûr,Très sûr,All
Q6,,,,,,
Marine Le Pen,1.78,2.07,4.04,4.93,37.67,50.49
Emmanuel Macron,1.18,1.08,4.34,2.86,22.39,31.85
,5.92,1.48,3.25,2.56,4.44,17.65
All,8.88,4.64,11.64,10.36,64.50,100.00


By normalizing these numbers per response category in Q5, we see that Macron and MLP voters are almost equally sure of their vote, with MLP edging Macron out, although I would not think this is a significant difference. 

In [21]:
pd.crosstab(index=dff['Q6'].replace({"Blanc":""}),columns=dff['Q5'].replace({"":"Autre (veuillez préciser)"}), \
            normalize='index',margins=True).apply(lambda r:r*100)\
            .round(decimals=2)


Q5,Autre (veuillez préciser),Cela dépend du candidat,Je ne sais pas,Pas sûr,Très sûr
Q6,,,,,
,33.52,8.38,18.44,14.53,25.14
Emmanuel Macron,3.72,3.41,13.62,8.98,70.28
Marine Le Pen,3.52,4.10,8.01,9.77,74.61
All,8.88,4.64,11.64,10.36,64.50


In [22]:
comparison=["Q7","Q11"]
print(comparison[0],"--",translated_qn[comparison[0]])
print(comparison[1],"--",translated_qn[comparison[1]])
temp=pd.crosstab(index=dff['Q7'],columns=dff['Q11'], \
            normalize=True,margins=True).apply(lambda r:r*100, axis=1)\
            .round(decimals=2)

index_all = temp.index.get_loc("All")
array_toReorder = list(temp["All"].argsort())
array_toReorder.reverse()
array_toReorder.remove(index_all)
array_toReorder.append(index_all)
new_index = [temp.index[x] for x in array_toReorder]
temp.reindex(new_index)
         

Q7 -- Who do you think will win this election?
Q11 -- Do you agree the person you think will win is the right choice for France?


Q11,,Non,Oui,All
Q7,,,,
Macron,0.69,9.57,27.51,37.77
,34.02,0.10,0.30,34.42
Le Pen,0.59,2.07,25.15,27.81
All,35.31,11.74,52.96,100.00


More people think that Macron will win. 

In [23]:
print(comparison[0],"--",translated_qn[comparison[0]])
print(comparison[1],"--",translated_qn[comparison[1]])

pd.crosstab(index=dff['Q7'],columns=dff['Q11'], \
            normalize='index',margins=True).apply(lambda r:r*100)\
            .round(decimals=2)
       

Q7 -- Who do you think will win this election?
Q11 -- Do you agree the person you think will win is the right choice for France?


Q11,,Non,Oui
Q7,,,
,98.85,0.29,0.86
Le Pen,2.13,7.45,90.43
Macron,1.83,25.33,72.85
All,35.31,11.74,52.96


But of those who think he will win, they are less positive on his presidency than those who think see a Le Pen presidency. 